# Feature Engineering Exploration

This notebook develops and validates the mathematical transformations that convert
raw match statistics into predictive features capturing temporal performance dynamics.

Mathematical Framework:
For player i at gameweek t, we construct feature vector X(i,t) that incorporates:
1. Rolling performance metrics with exponential decay weighting
2. Opponent-adjusted statistics normalized by defensive strength
3. Fixture context including upcoming difficulty and schedule density

The temporal ordering constraint requires that X(i,t) depends only on information
available at or before gameweek t, preventing information leakage.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

## Visualization configuration

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## CONFIGURATION

In [ ]:
BASE_DIR = Path.cwd().parent if 'Notebooks' in str(Path.cwd()) else Path.cwd()
PROCESSED_DIR = BASE_DIR / "data" / "processed"

print(f"Base Directory: {BASE_DIR}")
print(f"Processed Data Directory: {PROCESSED_DIR}")
print("\n" + "="*80 + "\n")

## SECTION 1: LOAD PREPROCESSED DATA

In [ ]:
df = pd.read_csv(PROCESSED_DIR / "fpl_unified_preprocessed.csv")

print(f"Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Temporal Range: {df['season'].min()} to {df['season'].max()}")
print(f"Gameweek Range: GW{df['GW'].min()} to GW{df['GW'].max()}")

# Convert kickoff_time to datetime for temporal operations
df['kickoff_time'] = pd.to_datetime(df['kickoff_time'])

# Sort by temporal order (critical for rolling calculations)
df = df.sort_values(['season', 'GW', 'element']).reset_index(drop=True)

print("\nData Types:")
print(df.dtypes.value_counts())

print("\n" + "="*80 + "\n")

## SECTION 2: ROLLING PERFORMANCE METRICS

Mathematical Foundation:

For metric m at time t, the exponentially weighted moving average is:
    EWMA(m,t) = Σ(k=1 to n) w(k) × m(t-k)

where weights follow exponential decay:
    w(k) = λ^k / Σ(j=1 to n) λ^j

The decay parameter λ ∈ (0,1) controls temporal weighting:
- λ close to 1: slow decay, equal weighting across window
- λ close to 0: rapid decay, recent observations dominate

We implement multiple windows to capture different temporal scales:
- 3-game window (λ=0.5): immediate recent form
- 5-game window (λ=0.6): medium-term trajectory  
- Season-long window: baseline quality assessment

In [ ]:
def calculate_rolling_stats(df: pd.DataFrame, 
                            metrics: list, 
                            windows: list = [3, 5]) -> pd.DataFrame:
    """
    Calculates rolling statistics for specified metrics across multiple windows.
    
    Implementation uses pandas rolling with min_periods=1 to handle early-season
    cases where insufficient historical data exists. This ensures every observation
    receives a feature value, though early-season features have higher uncertainty.
    
    Parameters
    ----------
    df : pd.DataFrame
        Sorted dataframe with temporal ordering
    metrics : list
        Performance metrics to aggregate
    windows : list
        Window sizes in number of gameweeks
        
    Returns
    -------
    pd.DataFrame
        Dataframe with added rolling statistic columns
    """
    df_rolled = df.copy()
    
    # Group by player to ensure rolling stats don't cross player boundaries
    for metric in metrics:
        if metric not in df.columns:
            print(f"Warning: {metric} not found in dataframe, skipping...")
            continue
            
        for window in windows:
            col_name = f'{metric}_roll_{window}gw'
            
            # Calculate rolling mean per player
            df_rolled[col_name] = (
                df_rolled.groupby('element')[metric]
                .rolling(window=window, min_periods=1)
                .mean()
                .reset_index(level=0, drop=True)
            )
    
    return df_rolled


# Define metrics to track
performance_metrics = [
    'total_points',
    'minutes', 
    'goals_scored',
    'assists',
    'clean_sheets',
    'ict_index',
    'influence',
    'creativity', 
    'threat',
    'bps'
]

# Add expected goals metrics for seasons where available
xg_metrics = ['expected_goals', 'expected_assists', 'expected_goal_involvements']
available_xg = [m for m in xg_metrics if m in df.columns]
performance_metrics.extend(available_xg)

print(f"Computing rolling statistics for {len(performance_metrics)} metrics...")
print(f"Windows: {[3, 5]} gameweeks")

df = calculate_rolling_stats(df, performance_metrics, windows=[3, 5])

print(f"\nNew columns added: {[c for c in df.columns if '_roll_' in c][:5]}...")
print(f"Total rolling features: {len([c for c in df.columns if '_roll_' in c])}")

# Validation: Check for NaN patterns in rolling features
print("\nValidation - Missing values in rolling features (sample):")
rolling_cols = [c for c in df.columns if '_roll_' in c][:5]
for col in rolling_cols:
    n_missing = df[col].isna().sum()
    print(f"{col:40s}: {n_missing:6d} missing ({n_missing/len(df)*100:5.2f}%)")

print("\n" + "="*80 + "\n")

### FORENSIC FEATURE AUDIT & CLEANUP

In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def forensic_cleanup_pipeline(df_in):
    """
    1. THE PURGE: Removes market noise features.
    2. THE INJECTION: Adds fixture context (Home/Away interaction).
    3. THE FILTER: Identifies and removes highly collinear features.
    """
    df_clean = df_in.copy()
    
    # --- 1. THE PURGE (Remove Market Noise) ---
    # These features cause feedback loops (high price -> high selection -> high price)
    # We want to predict points based on PERFORMANCE, not POPULARITY.
    noise_features = [
        'value', 'selected', 'transfers_in', 'transfers_out', 
        'transfers_balance', 'own_goals', 'penalties_missed', 'penalties_saved'
    ]
    dropped_noise = [c for c in noise_features if c in df_clean.columns]
    df_clean = df_clean.drop(columns=dropped_noise)
    print(f"[PURGE] Removed {len(dropped_noise)} market noise features: {dropped_noise}")

    # --- 2. THE INJECTION (Fixture Difficulty Context) ---
    # We approximate difficulty using Home/Away form. 
    # Logic: Performing well at HOME is standard. Performing well AWAY is a signal of class.
    # If 'was_home' and rolling points exist, create interaction features.
    
    if 'was_home' in df_clean.columns and 'total_points_roll_5gw' in df_clean.columns:
        # Convert boolean to int if necessary
        is_home = df_clean['was_home'].astype(int)
        
        # Feature: Form when playing at Home
        df_clean['form_at_home'] = df_clean['total_points_roll_5gw'] * is_home
        
        # Feature: Form when playing Away (High value here is a very strong signal)
        df_clean['form_away'] = df_clean['total_points_roll_5gw'] * (1 - is_home)
        
        print("[INJECTION] Created 'form_at_home' and 'form_away' interaction features.")
    
    # --- 3. THE FILTER (Remove Multicollinearity) ---
    # Mathematical Goal: Reduce X dimension where Corr(Xi, Xj) > 0.95
    
    # Select only numeric features for correlation check
    numeric_df = df_clean.select_dtypes(include=[np.number])
    
    # Exclude metadata and target from being dropped
    protected_cols = ['season', 'GW', 'element', 'total_points', 'minutes']
    candidates = [c for c in numeric_df.columns if c not in protected_cols]
    
    # Compute correlation matrix
    corr_matrix = df_clean[candidates].corr().abs()
    
    # Select upper triangle
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    
    # Perform the drop
    df_clean = df_clean.drop(columns=to_drop)
    
    print(f"[FILTER] Identified {len(to_drop)} redundant features (Corr > 0.95).")
    if len(to_drop) > 0:
        print(f"         Examples: {to_drop[:5]}...")
    
    return df_clean

# --- EXECUTE PIPELINE ---
print(f"Original Shape: {df.shape}")
df_model_ready = forensic_cleanup_pipeline(df)
print(f"Clean Shape:    {df_model_ready.shape}")

# --- OPTIONAL: SAVE FOR MODELING ---
output_path = PROCESSED_DIR / "fpl_model_ready.csv"
df_model_ready.to_csv(output_path, index=False)
print(f"\nSaved clean dataset to: {output_path}")

## SECTION 3: VISUALIZE ROLLING STATISTICS

In [ ]:
# Select a high-profile player for visualization
# Filter to players with substantial appearances
player_games = df.groupby('element').size()
active_players = player_games[player_games >= 20].index

# Get a sample player
if len(active_players) > 0:
    sample_player_id = active_players[232]
    player_data = df[df['element'] == sample_player_id].copy()
    player_name = player_data['name'].iloc[0] if 'name' in player_data.columns else f"Player {sample_player_id}"
    
    print(f"Visualizing rolling statistics for: {player_name}")
    print(f"Total appearances: {len(player_data)}")
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Rolling Performance Metrics - {player_name}', fontsize=14, fontweight='bold')
    
    # Plot 1: Total Points
    ax = axes[0, 0]
    ax.plot(player_data['GW'], player_data['total_points'], 'o-', alpha=0.3, label='Actual')
    ax.plot(player_data['GW'], player_data['total_points_roll_3gw'], '-', linewidth=2, label='3-GW Average')
    ax.plot(player_data['GW'], player_data['total_points_roll_5gw'], '-', linewidth=2, label='5-GW Average')
    ax.set_xlabel('Gameweek')
    ax.set_ylabel('Total Points')
    ax.set_title('Total Points - Rolling Averages')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: ICT Index
    ax = axes[0, 1]
    ax.plot(player_data['GW'], player_data['ict_index'], 'o-', alpha=0.3, label='Actual')
    ax.plot(player_data['GW'], player_data['ict_index_roll_3gw'], '-', linewidth=2, label='3-GW Average')
    ax.plot(player_data['GW'], player_data['ict_index_roll_5gw'], '-', linewidth=2, label='5-GW Average')
    ax.set_xlabel('Gameweek')
    ax.set_ylabel('ICT Index')
    ax.set_title('ICT Index - Rolling Averages')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Minutes Played
    ax = axes[1, 0]
    ax.plot(player_data['GW'], player_data['minutes'], 'o-', alpha=0.3, label='Actual')
    ax.plot(player_data['GW'], player_data['minutes_roll_3gw'], '-', linewidth=2, label='3-GW Average')
    ax.plot(player_data['GW'], player_data['minutes_roll_5gw'], '-', linewidth=2, label='5-GW Average')
    ax.set_xlabel('Gameweek')
    ax.set_ylabel('Minutes')
    ax.set_title('Minutes Played - Rolling Averages')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Expected Goals (if available)
    ax = axes[1, 1]
    if 'expected_goals' in player_data.columns and not player_data['expected_goals'].isna().all():
        ax.plot(player_data['GW'], player_data['expected_goals'], 'o-', alpha=0.3, label='Actual xG')
        if 'expected_goals_roll_3gw' in player_data.columns:
            ax.plot(player_data['GW'], player_data['expected_goals_roll_3gw'], '-', linewidth=2, label='3-GW Avg')
        ax.set_xlabel('Gameweek')
        ax.set_ylabel('Expected Goals')
        ax.set_title('Expected Goals - Rolling Averages')
        ax.legend()
    else:
        ax.text(0.5, 0.5, 'Expected Goals data not available\nfor this player/season', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(BASE_DIR / 'notebooks' / 'figures' / 'rolling_stats_example.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nVisualization complete. Notice how rolling averages smooth volatility.")
    print("The 3-GW average responds quickly to form changes.")
    print("The 5-GW average provides more stable trend assessment.")

else:
    print("No players with sufficient appearances found for visualization.")

print("\n" + "="*80 + "\n")

## SECTION 4: OPPONENT STRENGTH INDICES

Mathematical Foundation:

Team defensive strength at gameweek t:
    DS(team,t) = Σ(k in recent matches) goals_conceded(k) / n_matches

Opponent adjustment factor for player i facing team j:
    α(i,j,t) = DS(j,t) / DS_league_avg(t)

Adjusted metric:
    m_adjusted(i,t) = m(i,t) × α(i,j,t)

This normalization isolates player skill from match difficulty. A midfielder
with 50 creativity against Manchester City (strong defense) provides more
predictive signal than 50 creativity against a relegation candidate.

In [ ]:
def calculate_team_defensive_strength(df: pd.DataFrame, window: int = 5) -> pd.DataFrame:
    """
    Calculates rolling defensive strength for each team.
    
    Defensive strength measures goals conceded per match over recent window.
    Lower values indicate stronger defenses. We use rolling window to capture
    current form rather than season-long averages that may be stale.
    
    Parameters
    ----------
    df : pd.DataFrame
        Match-level data with team and goals_conceded
    window : int
        Number of recent matches to average
        
    Returns
    -------
    pd.DataFrame
        Dataframe with team defensive strength indices
    """
    # Calculate goals conceded per team per gameweek
    team_defense = (
        df.groupby(['season', 'GW', 'team'])['goals_conceded']
        .mean()  # Average across players (same value for all team members)
        .reset_index()
    )
    
    # Calculate rolling defensive strength per team
    team_defense = team_defense.sort_values(['team', 'season', 'GW'])
    team_defense['def_strength'] = (
        team_defense.groupby('team')['goals_conceded']
        .rolling(window=window, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )
    
    return team_defense[['season', 'GW', 'team', 'def_strength']]


# Calculate team defensive indices
print("Computing team defensive strength indices...")
team_defense_df = calculate_team_defensive_strength(df, window=5)

print(f"\nDefensive strength calculated for {team_defense_df['team'].nunique()} teams")
print("\nSample defensive strengths (lower = stronger defense):")
print(team_defense_df.groupby('team')['def_strength'].mean().sort_values().head(10))

# Calculate league average defensive strength per gameweek
league_avg_defense = (
    team_defense_df.groupby(['season', 'GW'])['def_strength']
    .mean()
    .reset_index()
    .rename(columns={'def_strength': 'league_avg_def'})
)

print("\n" + "="*80 + "\n")

## SECTION 5: OPPONENT-ADJUSTED METRICS

We merge opponent defensive strength into player records and calculate
adjustment factors. This allows us to normalize performance metrics by
the quality of opposition faced.

In [ ]:
if 'opponent_team' in df.columns:
    # Create team ID to name mapping
    team_id_map = df[['team', 'opponent_team']].drop_duplicates()
    # This is tricky - opponent_team is numeric ID, need proper mapping
    # For now, we'll skip opponent adjustment and note it needs team ID mapping
    
    print("Note: Opponent adjustment requires proper team ID to name mapping.")
    print("This will be implemented in the production feature engineering module.")
    print("The mathematical framework is established above.")

print("\n" + "="*80 + "\n")

## SECTION 6: FIXTURE DENSITY FEATURES

Mathematical Foundation:

Fixture density quantifies upcoming match congestion, which affects rotation
probability and minutes expectation. For gameweek t, we count fixtures in
forward-looking windows:

    density_7d(t) = |{fixtures in [t, t+7 days]}|
    density_14d(t) = |{fixtures in [t, t+14 days]}|

High fixture density increases rotation risk, particularly for players at
elite clubs with deep squads.

## SECTION 7: POSITION-SPECIFIC FEATURES

Different positions exhibit distinct scoring patterns that require specialized
feature engineering. We analyze position-specific distributions to inform
feature construction.

In [ ]:
# Analyze position-specific statistics
print("Position-Specific Performance Distributions:\n")

position_stats = df.groupby('position').agg({
    'total_points': ['mean', 'std', 'median'],
    'minutes': ['mean', 'median'],
    'goals_scored': 'mean',
    'assists': 'mean',
    'clean_sheets': 'mean',
    'ict_index': 'mean'
}).round(2)

print(position_stats)

# Visualize position-specific distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Position-Specific Performance Distributions', fontsize=14, fontweight='bold')

# Points distribution by position
ax = axes[0, 0]
df.boxplot(column='total_points', by='position', ax=ax)
ax.set_title('Total Points by Position')
ax.set_xlabel('Position')
ax.set_ylabel('Total Points')
plt.sca(ax)
plt.xticks(rotation=0)

# ICT Index by position
ax = axes[0, 1]
df.boxplot(column='ict_index', by='position', ax=ax)
ax.set_title('ICT Index by Position')
ax.set_xlabel('Position')
ax.set_ylabel('ICT Index')
plt.sca(ax)
plt.xticks(rotation=0)

# Minutes by position
ax = axes[1, 0]
df.boxplot(column='minutes', by='position', ax=ax)
ax.set_title('Minutes Played by Position')
ax.set_xlabel('Position')
ax.set_ylabel('Minutes')
plt.sca(ax)
plt.xticks(rotation=0)

# Goals + Assists by position
ax = axes[1, 1]
df['goal_contributions'] = df['goals_scored'] + df['assists']
df.boxplot(column='goal_contributions', by='position', ax=ax)
ax.set_title('Goal Contributions by Position')
ax.set_xlabel('Position')
ax.set_ylabel('Goals + Assists')
plt.sca(ax)
plt.xticks(rotation=0)

plt.tight_layout()
plt.savefig(BASE_DIR / 'notebooks' / 'figures' / 'position_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nKey Observations:")
print("- Midfielders show highest ICT index variation (offensive and defensive roles)")
print("- Forwards have highest scoring variance (feast or famine distribution)")
print("- Defenders show most consistent minutes (less rotation)")
print("- Goalkeepers have unique clean sheet dependency")

print("\n" + "="*80 + "\n")

## SECTION 8: FEATURE CORRELATION ANALYSIS

Examine correlations among engineered features to identify redundancy.
High correlation indicates features capture overlapping information.

In [ ]:
# Select rolling features for correlation analysis
rolling_features = [c for c in df.columns if '_roll_' in c and '3gw' in c]
core_features = ['total_points', 'ict_index', 'minutes', 'goals_scored', 'assists']

analysis_cols = core_features + rolling_features[:8]  # Limit for readability
analysis_cols = [c for c in analysis_cols if c in df.columns]

# Calculate correlation matrix on non-null subset
corr_df = df[analysis_cols].corr()

# Visualize
plt.figure(figsize=(12, 10))
sns.heatmap(corr_df, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix - Core and Rolling Statistics', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(BASE_DIR / 'notebooks' / 'figures' / 'feature_correlations.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nCorrelation insights:")
print("- Strong correlation between total_points and its rolling averages (expected)")
print("- ICT components show moderate correlation (capture different aspects)")
print("- Rolling windows of same metric highly correlated (expected temporal smoothing)")

print("\n" + "="*80 + "\n")

## SECTION 9: TARGET VARIABLE ENGINEERING

Our prediction target is cumulative points over next 3-4 gameweeks.
We construct this forward-looking target by summing future performance.

Mathematical formulation:
    Y(i,t) = Σ(k=1 to h) total_points(i, t+k)

where h ∈ {3,4} is the forecast horizon.

CRITICAL: This target cannot be used for training at gameweek t. It represents
future information. We construct it here for validation purposes.

In [ ]:
def create_forward_target(df: pd.DataFrame, horizon: int = 3) -> pd.DataFrame:
    """
    Creates forward-looking cumulative points target.
    
    This target represents the sum of points over the next `horizon` gameweeks.
    It can only be used for validation on past data or for current prediction
    where we are forecasting the unknown future.
    
    Parameters
    ----------
    df : pd.DataFrame
        Match data sorted temporally
    horizon : int
        Number of future gameweeks to sum
        
    Returns
    -------
    pd.DataFrame
        Dataframe with forward target column
    """
    df_target = df.copy()
    
    # Calculate cumulative future points per player
    df_target[f'target_points_{horizon}gw'] = (
        df_target.groupby('element')['total_points']
        .rolling(window=horizon, min_periods=1)
        .sum()
        .shift(-horizon + 1)  # Shift to align with current gameweek
        .reset_index(level=0, drop=True)
    )
    
    return df_target

# Create targets for different horizons
print("Creating forward-looking targets for horizons: 3, 4 gameweeks...")

df = create_forward_target(df, horizon=3)
df = create_forward_target(df, horizon=4)

# Validate target construction
sample_player = df[df['element'] == active_players[0]].head(10)
print("\nSample Target Construction (first 10 gameweeks of sample player):")
print(sample_player[['GW', 'total_points', 'target_points_3gw', 'target_points_4gw']])

print("\nTarget variable statistics:")
for h in [3, 4]:
    target_col = f'target_points_{h}gw'
    print(f"\n{target_col}:")
    print(df[target_col].describe())

print("\n" + "="*80 + "\n")

## SECTION 10: SUMMARY AND NEXT STEPS

In [ ]:
print("\nFeatures Constructed:")
print(f"1. Rolling performance metrics: {len([c for c in df.columns if '_roll_' in c])} features")
print("2. Position-specific analysis completed")
print("3. Target variables: target_points_3gw, target_points_4gw")

print("\nFeatures Requiring Production Implementation:")
print("1. Opponent strength adjustment (needs team ID mapping)")
print("2. Fixture density metrics (needs fixture schedule API)")
print("3. Price-performance interaction features")
print("4. Lag features (previous GW performance)")

print("\nCurrent Dataset Dimensions:")
print(f"Rows: {len(df):,}")
print(f"Columns: {len(df.columns)}")

print("\nData Quality Check:")
rolling_cols = [c for c in df.columns if '_roll_' in c]
for col in rolling_cols[:3]:
    n_missing = df[col].isna().sum()
    print(f"{col:40s}: {n_missing:6d} missing ({n_missing/len(df)*100:5.2f}%)")

print("\n" + "="*80)
print("FEATURE ENGINEERING EXPLORATION COMPLETE")
print("="*80)

In [ ]:
# Save engineered features for next phase
output_path = PROCESSED_DIR / "fpl_features_exploratory.csv"
df.to_csv(output_path, index=False)
print(f"\nEngineered features saved to: {output_path}")